In [ ]:
#package to read delimited (csv) files
using DelimitedFiles

In [ ]:
#read Ebola virus data
ev_data = DelimitedFiles.readdlm("wikipediaEVDraw.csv", ',')


In [ ]:
#convert date string to DateTime format
using Dates
Dates.DateTime(ev_data[1,1], "d u y")

In [ ]:
#converting col 1 to DateTime type as shown above 
col1 = ev_data[:,1]
for i=1:length(col1)
    col1[i] = Dates.DateTime(col1[i], "d u y")
end 

In [ ]:
#create a data giving time in days since 22 Mar 2014 (Rata Die days format)
#it gives number of days since 1 Jan 0001
Dates.datetime2rata(col1[1])


In [ ]:
#create a function to express number of days since epidemic day zero, which is col1[54], which is 22 March 2014
days_since_mar22(x) = Dates.datetime2rata(x)-Dates.datetime2rata(col1[54])
epidays = Array{Int64}(undef, 54)
for i = 1:length(col1)
    epidays[i] = days_since_mar22(col1[i])
end 

In [ ]:
#export converted data 
ev_data[:,1]=epidays
DelimitedFiles.writedlm("wikipediaEVDdatesconverted.csv", ev_data, ",")

In [ ]:
using Pkg
Pkg.add("Plots")
using Plots 

In [ ]:
#use GR backend for plots 
gr()

In [ ]:
plot(ev_data[:,1], ev_data[:,2])

In [ ]:
plot(ev_data[:,1], ev_data[:,2], linetype = :scatter, marker = :diamond)

In [ ]:
plot(ev_data[:, 1], ev_data[:, 2],
    title="West Africa Ebola epidemic, total cases",
    xlabel="Days since 22 March 2024",
    ylabel="Total cases to data (three countries)",
    marker=(:diamond, 8, "orange"), #paranthesis to group the marker attributes into a composite of attributes
    line=(:path, "gray"),
    legend=false,
    grid=false)

In [ ]:
#save plot 
savefig("WAfricaEVD")#by default png 
savefig("WAfricaEVD.pdf")#by default png 

In [ ]:
#fill in missing values 
rows, cols = size(ev_data)
for j = 1:cols
    for i = 1:rows
        if !isnumeric(string(ev_data[i, j])[1])
            ev_data[i, j] = 0
        end
    end
end

In [ ]:
#Array slicing 
ev_data[end-9:end, :] #gives last 10 rows
epidays = ev_data[:,1]
evCasesByCountry = ev_data[:,[4,6,8]]

In [ ]:
#plot 
plot(epidays, evCasesByCountry[:, 1])
plot!(epidays, evCasesByCountry[:, 2])
plot!(epidays, evCasesByCountry[:, 3])
plot!(title="West Africa Ebola epidemic, total cases",
    xlabel="Days since 22 March 2024",
    ylabel="Total cases to data (three countries)",
    marker=([:octagon :star :square], 8), #paranthesis to group the marker attributes into a composite of attributes
    line=(:scatter),
    label=["Guinea" "Liberia" "Sierra Leone"],
    legend=:topleft)


In [ ]:
plot(epidays, [evCasesByCountry[:,1] evCasesByCountry[:,2] evCasesByCountry[:,3]])

In [ ]:
size(ev_data[end-3:end, :])
size(ev_data)

***SIR (Susceptible, Infected and Removed) Models***

We make the following assumptions.

- Before the epidemic starts, there is a population at risk of catching disease 
- The disease spreads from infected to susceptible people
- After a while, people are no longer infected
- Once infected, people are never infected again
- We ignore allother differences between people 

$ S $ (Susceptible) for number of people still at risk of the disease. $S$ is a function of time, $S(t)$. The number $S(0)$ is the number of susceptibles at the start of the epidemic.
The number of people infected at time $t$ is $I(t)$. We assume $I(0) > 0$, otherwise epidemic can't start. People who are previously infected, but no longer infectious are called Removed ($R$)

***SIR Model equations***

We will model $S$, $I$, and $R$ in discrete time steps. In fact we will use the same length of time step $dt$ for every step. We denote $t_i$, the time after $i$ steps. We assume $t_0=0$ adn $t_i = i dt$. The model steps from 
$t_i$ to $t_{i+1}$, the three equations all have the form 

new value = old value + gains - losses

For susceptibles, there is only a loss term. The simple SIR models come from the so-called "law of mass action". This law holds that all susceptibles have an equal chance of meeting an infected person. And the same is assumed for infected people. Then the number of meetings of infected and susceptible is proportional to the product $SI$. We use the symbol $\lambda$ for the constant of proportionality and we interpret $\lambda S I$ as the rate at which suscpetible becomes infected. This is per day, so actual loss over the time step $dt$ is actually $\lambda S I$ so the equation for $S$ is 

1... $S(t_{i+1}) = S(t_i) - \lambda S(t_i) I(t_i)dt$

The loss rate for infected people is just a constant propbability of recovery per unit time. So a constant fraction $\gamma$ is removed from the infected per day. The gain of infecteds must be exactly the susceptible who becomes infected, so the equation for $I$ is 

2... $I(t_{i+1}) = I(t_i) + \lambda S(t_i) I(t_i)dt - \gamma I(t_i)dt$

In an SIR model, the removed population equation has only a gain term, no loss term. What is lost from infected is gained by removeds, so the equation for $R$ is 

3... $R(t_{i+1}) = R(t_i) + \gamma I(t_i)dt$










In [ ]:
function updateSIR(popnvector, dt, lambda, gam)
    susceptibles = popnvector[1]
    infecteds = popnvector[2]
    removeds = popnvector[3]

    newS = susceptibles - lambda * susceptibles * infecteds * dt
    newI = infecteds + lambda * susceptibles * infecteds * dt - gam * infecteds * dt
    newR = removeds + gam * infecteds * dt

    return [newS newI newR]
end

In [ ]:
dt = 0.5 #two steps per day 
lambda = 1/200; gam = 1/10 #since gamma is julia function we use gam instead 

s, i, r = 1000, 10, 20 
vec = [s i r]
updateSIR(vec, dt, lambda, gam)

In [ ]:
# we will use number of days as 610 (slightly more than wikipedia data
lambda = 1/20000 #infection rate (assumes rate per day)
gam = 1/10 #recovery rate 
dt = 0.5 
s0 = 10000.0
i0 = 4.0 
r0 = 0.0 
tfinal = 610 
nsteps = round(Int64, tfinal/dt)
resultvals = Array{Float64}(undef, nsteps + 1, 3)
timevec = Array{Float64}(undef, nsteps + 1)

resultvals[1,:] = [s0, i0, r0]
timevec[1] = 0.0

for step = 1:nsteps
    resultvals[step+1, :] = updateSIR(resultvals[step, :], dt, lambda, gam)
    timevec[step+1] = timevec[step] + dt 
end 

In [ ]:
plot(timevec, resultvals, title="Example of SIR", xlabel="Epidemic day", ylabel="Polulation size", label=["Susceptibles" "Infecteds" "Removeds"], legend=:topright)

The parameter $\lambda$ models the likelihood that when an infected person meets a susceptible person, the susceptible person becomes infected. 

The parameter $\gamma$ models the rate at which infected people become removed

For a given value of $\lambda$ and $\gamma$, there is no epidemic if $S(0)$ is big enough. Thre reason is for an epidemic to happen, the infection must spread. This means, more people must be getting infected than are recovering. That is, gain minus the loss of infected must be positive

$\lambda S I - \gamma I \gt 0 $ 

or 

$\lambda S - \gamma \gt 0 $

So that number of infecteds can increase only when 

$ S \gt \dfrac{\gamma}{\lambda}$

This implies that if initial population is below threshold $\left( S(0) \lt \dfrac{\gamma}{\lambda} \right)$, then the number of infecteds, even if initially large, will decrease. That is, in a small enough population, any initial infection simply dies out. On the other hand, if $ S(0) \gt \dfrac{\gamma}{\lambda}$, the number of infecteds will increase, at least for a while. 

The loss rate of infected is given by $\gamma I$, which means that every day, the fraction of infected people becoming removed is $\gamma$

if $\gamma = 1/10$, every day a tenth of ill people will recover. If we assume that every person is ill for 10 days, and as many people are getting ill as recovering, then a tenth of all ill people recover every day. 

In other words, it makes sense to think of the duration of one person's illness as $1 / \gamma$. The duration of EVD is approximately three weeks. This means we may estimate that $1/\gamma \approx 21$. And set $\gamma = 0.05$. Although it is crude to think that they are equally infectious every day of these three weeks

In [ ]:
lambda = 0.0005 #infection rate (assumes rate per day)
gam = 1/20 #recovery rate 
dt = 0.5 
s0 = 2000.0
i0 = 4.0 
r0 = 0.0 
tfinal = 610 
nsteps = round(Int64, tfinal/dt)
resultvals = Array{Float64}(undef, nsteps + 1, 3)
timevec = Array{Float64}(undef, nsteps + 1)

resultvals[1,:] = [s0, i0, r0]
timevec[1] = 0.0

for step = 1:nsteps
    resultvals[step+1, :] = updateSIR(resultvals[step, :], dt, lambda, gam)
    timevec[step+1] = timevec[step] + dt 
end 

In [ ]:
#Good way to illustrate threshold phenomena is plotting I vs S in phase plane 
plot(resultvals[:,1],resultvals[:,2], title="I vs S", xlabel="Susceptibles", ylabel="Infecteds")

#Susceptibles start at 2000 and rapidly increase and peak over 1500 and drops very fast 

Lets now estimate $\lambda$. At the beginning of the epidemic, one could not safely say that any person in three countries concerned would never be exposed to the Ebola virus, we should assume that the susceptible population was the sum of the populations of Guinea, Liberia and Sierra Leone. The population is 

$S(0) = 22 \times 10^6$

Now we use the threshold, which we write as $S^* = \dfrac{\gamma}{\lambda}$, so that $\lambda = \dfrac{\gamma}{S^*}$. Assuming that our $S(0)$ is far larger that $S^*$, because epidemic was so severe, let us set 

$S^* = S(0)$ 

So $\lambda = 0.05/(0.1 \times 2.2 \times 10^6) \approx 2.3 \times 10^{-7}$. Using these values 

In [ ]:
lambda = 2.3e-8 #infection rate (assumes rate per day)
gam = 1/20 #recovery rate 
dt = 0.5 
s0 = 22.0e6
i0 = 4.0 
r0 = 0.0 
tfinal = 610 
nsteps = round(Int64, tfinal/dt)
resultvals = Array{Float64}(undef, nsteps + 1, 3)
timevec = Array{Float64}(undef, nsteps + 1)

resultvals[1,:] = [s0, i0, r0]
timevec[1] = 0.0

for step = 1:nsteps
    resultvals[step+1, :] = updateSIR(resultvals[step, :], dt, lambda, gam)
    timevec[step+1] = timevec[step] + dt 
end 

plot(resultvals[:,1],resultvals[:,2], title="I vs S", xlabel="Susceptibles", ylabel="Infecteds")

**Total cases data: observation vs model**
The total cases to date is the number of people who have ever been infected. But every such person is either in the infected group currently or has moved to the removed group. So the total number of cases at the time $t$ is 

$C(t) = I(t) + R(t)$

We will use $W(t)$ to represent the reported number of cases from Wikipedia 

In [ ]:
C = resultvals[:,2] + resultvals[:,3]
plot(timevec, C, label = "Model values", xlabel="Epidemic day", ylabel="Number of cases to date", title="Model vs data")
plot!(ev_data[:,1], ev_data[:,2], legend=:right, line=:scatter, label="Reported number of cases")

Model results doesn't make sense. If we adjust $S(0)$, $\lambda$ and $\gamma$ based on observed values, we note that we can get model close to reported values by having the following values 

$S(0) = 10^5$, $I(0)=20$, $\gamma=1/8$ and $\lambda = 1.46 \times 10^{-6}$

In [ ]:
lambda = 1.47e-6 #infection rate (assumes rate per day)
gam = 1/8 #recovery rate 
dt = 0.5 
s0 = 1e5
i0 = 20.0 
r0 = 0.0 
tfinal = 610 
nsteps = round(Int64, tfinal/dt)
resultvals = Array{Float64}(undef, nsteps + 1, 3)
timevec = Array{Float64}(undef, nsteps + 1)

resultvals[1,:] = [s0, i0, r0]
timevec[1] = 0.0

for step = 1:nsteps
    resultvals[step+1, :] = updateSIR(resultvals[step, :], dt, lambda, gam)
    timevec[step+1] = timevec[step] + dt 
end 
C = resultvals[:,2] + resultvals[:,3]
plot(timevec, C, label = "Model values", xlabel="Epidemic day", ylabel="Number of cases to date", title="Model vs data")
plot!(ev_data[:,1], ev_data[:,2], legend=:right, line=:scatter, label="Reported number of cases")